In [4]:
import numpy as np
import pandas as pd
import json
import tweepy
import seaborn as sns
import os
import requests as req
import calendar
import pytz
import time
import got3

from datetime import datetime, date, timedelta

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()



In [5]:
api_dir = os.path.dirname(os.path.dirname(os.path.realpath('keys')))
file_name = os.path.join(api_dir + "//keys", "api_keys.json")
data = json.load(open(file_name))

gkey = data['google_api_key']
consumer_key = data['twitter_consumer_key']
consumer_secret = data['twitter_consumer_secret']
access_token = data['twitter_access_token']
access_token_secret = data['twitter_access_token_secret']


In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())



In [7]:
#coin list by symbol in descending order of market cap
#bitcoin - ethereum - ripple - litecoin - bitcoin cash - eos - neo - digit dao - tronix - ethereum classic  
coin_list = ['BTC', 'ETH', 'XRP', 'LTC', 'BCH', 'EOS', 'NEO', 'DGD', 'TRX', 'ETC']



In [8]:
#URLs
social_stats_url = "https://www.cryptocompare.com/api/data/socialstats/?id="

coin_list_url = "https://www.cryptocompare.com/api/data/coinlist/"

coin_historical_url = "https://min-api.cryptocompare.com/data/pricehistorical?fsym="
coin_historical_url_second = "&tsyms=USD&ts="



In [9]:
#variables 
coin_list_response = []

coin_data_df = pd.DataFrame(columns=["id", "symbol", "coin_name", "coin_price",  
                                     "algorithm", "date",
                                     "total_supply", "total_free_float"
                                   ])


coin_data_df.set_index("symbol")
coin_data_df.head()

,id,symbol,coin_name,coin_price,algorithm,date,total_supply,total_free_float


In [10]:
#date
year = 2018
month = 1
day = 1

In [11]:
def GetCoinList():
    response = req.get(coin_list_url).json()

    count = 0
    for each in response["Data"]:
        count = count + 1
    
    print(count)
    return response


def GetCoinId(symbol):
    for each in coin_list_response["Data"]:
        row = coin_list_response["Data"][symbol]
        if row["Name"] == symbol:
            return row["Id"]


def GetCoinName(symbol):
    for each in coin_list_response["Data"]:
        row = coin_list_response["Data"][symbol]
        if row["Name"] == symbol:
            return row["CoinName"].lower()

        
def GetAlgorithm(symbol):
    for each in coin_list_response["Data"]:
        row = coin_list_response["Data"][symbol]
        if row["Name"] == symbol:
            return row["Algorithm"]
        
        
def GetHistoricalPrice(symbol, utctime):
    #url to get historical price
    url = coin_historical_url + symbol + coin_historical_url_second + str(utctime)
    #get the historical price
    return req.get(url).json()
    

def GetSocialStats(coin_id):
    url = social_stats_url + coin_id 
    return req.get(url).json()


def GetUtcTime(year,month,day):
    utc = calendar.timegm(datetime(year,month,day).utctimetuple())
    #move to the next day
    next_date = date(year,month,day) + timedelta(1)
    year = next_date.year
    month = next_date.month
    day = next_date.day
    return utc
    
def SetValues(symbol,row):
    coin_data_df.set_value(symbol, "id", int(row["id"]))
    coin_data_df.set_value(symbol, "symbol", row["symbol"])
    
    new_date = date(year,month,day).strftime("%Y-%m-%d")  
    coin_data_df.set_value(symbol, "date", new_date)

    coin_data_df.set_value(symbol, "coin_name", row["coin_name"])
    coin_data_df.set_value(symbol, "coin_price", float(row["coin_price"]))
    coin_data_df.set_value(symbol, "algorithm", row["algorithm"])
    coin_data_df.set_value(symbol, "total_supply", int(row["total_supply"]))
    coin_data_df.set_value(symbol, "total_free_float", int(row["total_free_float"]))
    

    
def GetTweetsByDate():
    start = '2017-01-01 00:00:00'
    end   = '2018-01-01 00:00:00'

    sterms = '"lung cancer" OR "lung cancers" OR "lungcancer" OR "lungcancers" OR \
        "lung tumor" OR "lungtumor" OR "lung tumors" OR "lungtumors" OR "lung neoplasm"'

    # Items from 0 to 500,000 (which *should* cover all tweets)
    # Increase by 4,000 for each cycle (because 5000-6000 is over the Twitter rate limit)
    # Then wait for 20 min before next request (becaues twitter request wait time is 15min)

    counter2 = 0
    for tweet in tweepy.Cursor(api.search,q=sterms,since=start, until=end).items(999999999):  
        print(tweet["text"])
    
    

In [12]:
#get coin list
coin_list_response = GetCoinList()

#build coin data
count = 0
while count < 7:
    #get UTC time, one day at a time
    utc_time = calendar.timegm(datetime(year,month,day).utctimetuple())
    
    for symbol in coin_list:
        coin_id = GetCoinId(symbol)   
        coin_name = GetCoinName(symbol)

        price = GetHistoricalPrice(symbol,utc_time)
        coin_price = price[symbol]["USD"]
        
        algorithm = GetAlgorithm(symbol)
        
        row = {"id": coin_id, "coin_name": coin_name, "coin_price": coin_price,
               "algorithm": algorithm, "symbol": symbol, "date": utc_time,
               "total_supply": 0, "total_free_float": 0}

        SetValues(symbol,row)
        #    GetSocialStats(row["id"])

    count = count + 1
    
coin_data_df.head()

2112


,id,symbol,coin_name,coin_price,algorithm,date,total_supply,total_free_float
BTC,1182,BTC,bitcoin,13444.9,SHA256,2018-01-01,0,0
ETH,7605,ETH,ethereum,756.2,Ethash,2018-01-01,0,0
XRP,5031,XRP,ripple,2.05,N/A,2018-01-01,0,0
LTC,3808,LTC,litecoin,224.34,Scrypt,2018-01-01,0,0
BCH,202330,BCH,bitcoin cash / bcc,2319.12,SHA256,2018-01-01,0,0


In [13]:
GetCoinName("BTC")


'bitcoin'

In [14]:
coin_list_response = GetCoinList()
coin_list_response

2112


{'BaseImageUrl': 'https://www.cryptocompare.com',
 'BaseLinkUrl': 'https://www.cryptocompare.com',
 'Data': {'BTC': {'Algorithm': 'SHA256',
   'CoinName': 'Bitcoin',
   'FullName': 'Bitcoin (BTC)',
   'FullyPremined': '0',
   'Id': '1182',
   'ImageUrl': '/media/19633/btc.png',
   'Name': 'BTC',
   'PreMinedValue': 'N/A',
   'ProofType': 'PoW',
   'SortOrder': '1',
   'Sponsored': False,
   'Symbol': 'BTC',
   'TotalCoinSupply': '21000000',
   'TotalCoinsFreeFloat': 'N/A',
   'Url': '/coins/btc/overview'},
  'ETH': {'Algorithm': 'Ethash',
   'CoinName': 'Ethereum',
   'FullName': 'Ethereum (ETH)',
   'FullyPremined': '0',
   'Id': '7605',
   'ImageUrl': '/media/20646/eth_logo.png',
   'Name': 'ETH',
   'PreMinedValue': 'N/A',
   'ProofType': 'PoW',
   'SortOrder': '2',
   'Sponsored': False,
   'Symbol': 'ETH',
   'TotalCoinSupply': '0',
   'TotalCoinsFreeFloat': 'N/A',
   'Url': '/coins/eth/overview'},
  'LTC': {'Algorithm': 'Scrypt',
   'CoinName': 'Litecoin',
   'FullName': 'Litecoi

In [15]:
anid = GetCoinId("BTC")
int(anid)


1182

In [16]:
coin_list_response["Data"]["BTC"]


{'Algorithm': 'SHA256',
 'CoinName': 'Bitcoin',
 'FullName': 'Bitcoin (BTC)',
 'FullyPremined': '0',
 'Id': '1182',
 'ImageUrl': '/media/19633/btc.png',
 'Name': 'BTC',
 'PreMinedValue': 'N/A',
 'ProofType': 'PoW',
 'SortOrder': '1',
 'Sponsored': False,
 'Symbol': 'BTC',
 'TotalCoinSupply': '21000000',
 'TotalCoinsFreeFloat': 'N/A',
 'Url': '/coins/btc/overview'}

In [45]:
#calendar.timegm(datetime(2010,5,10).utctimetuple())
utc_time = calendar.timegm(datetime(year,month,day).utctimetuple())
type(utc_time)
datetime.utcfromtimestamp(float(utc_time))

datetime.datetime(2018, 1, 1, 0, 0)

In [18]:

newdate=date(2017,1,28)+timedelta(1)

another = date(2017,1,28)
print(newdate)
print(newdate.year)
print(newdate.month)
print(newdate.day)
#newdate.strftime("%Y-%m-%d")  

another.strftime("%Y-%m-%d")  


2017-01-29
2017
1
29


'2017-01-28'

In [19]:
social = GetSocialStats("3808")


In [20]:
coin_data_df["coin_name"]


BTC               bitcoin
ETH              ethereum
XRP                ripple
LTC              litecoin
BCH    bitcoin cash / bcc
EOS                   eos
NEO                   neo
DGD             digix dao
TRX                tronix
ETC      ethereum classic
Name: coin_name, dtype: object

In [21]:
row = {'id': '1182', 'coin_name': 'bitcoin', 'coin_price': 995.44, 'algorithm': '', 'total_supply': 0, 'total_free_float': 0}
type(row)
row["algorithm"]


''

In [22]:
utc_time = calendar.timegm(datetime(2017,1,28).utctimetuple())
price = GetHistoricalPrice("BTC",utc_time)
price


{'BTC': {'USD': 918.51}}

In [23]:
url = "https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&limit=60&aggregate=3&e=CCCAGG"
#req.get(url).json()

In [66]:
coin_market_cap_df = pd.DataFrame(columns=["name", "symbol", "price_usd", "market_cap_usd",
                                           "total_supply", "last_updated"])
coin_market_cap_df.set_index("symbol")

url = "https://api.coinmarketcap.com/v1/ticker"
res = req.get(url).json()
for each in res:
    coin_market_cap_df.set_value(each["symbol"], "symbol", each["symbol"])    
    coin_market_cap_df.set_value(each["symbol"], "name", each["name"])    
    coin_market_cap_df.set_value(each["symbol"], "price_usd", each["price_usd"])    
    coin_market_cap_df.set_value(each["symbol"], "market_cap_usd", each["market_cap_usd"])    
    coin_market_cap_df.set_value(each["symbol"], "total_supply", each["total_supply"])
    
    if each["last_updated"] is None:
        pass
    else:
        last_updated = datetime.utcfromtimestamp(float(each["last_updated"]))
        coin_market_cap_df.set_value(each["symbol"], "last_updated", last_updated.strftime("%Y-%m-%d"))    

coin_market_cap_df["market_cap_usd"] = coin_market_cap_df["market_cap_usd"].astype(float)
coin_market_cap_df = coin_market_cap_df.sort_values("market_cap_usd", ascending=False) 
coin_market_cap_df

    
    

,name,symbol,price_usd,market_cap_usd,total_supply,last_updated
BTC,Bitcoin,BTC,9009.01,1.517469e+11,16843900.0,2018-02-04
ETH,Ethereum,ETH,944.945,9.203471e+10,97396891.0,2018-02-04
XRP,Ripple,XRP,0.90311,3.522961e+10,99992725510.0,2018-02-04
BCH,Bitcoin Cash,BCH,1259.89,2.135268e+10,16948050.0,2018-02-04
ADA,Cardano,ADA,0.426598,1.106044e+10,31112483745.0,2018-02-04
LTC,Litecoin,LTC,161.276,8.879523e+09,55057933.0,2018-02-04
XLM,Stellar,XLM,0.425588,7.844113e+09,103649583897,2018-02-04
NEO,NEO,NEO,118.875,7.726875e+09,100000000.0,2018-02-04
EOS,EOS,EOS,9.65782,6.234971e+09,900000000.0,2018-02-04
XEM,NEM,XEM,0.608662,5.477958e+09,8999999999.0,2018-02-04


In [65]:
coin_market_cap_df = coin_market_cap_df.sort_values("symbol", ascending=True) 
coin_market_cap_df.head(80)


,name,symbol,price_usd,market_cap_usd,total_supply,last_updated
10MT,10M Token,10MT,0.00487771,NaN,10000000.0,2018-02-04
AAC,Acute Angle Cloud,AAC,0.188183,NaN,1000000000.0,2018-02-04
ABC,Alphabit,ABC,99.1973,NaN,210000000.0,2018-02-04
ACES,Aces,ACES,0.00075905,NaN,None,2018-02-04
ACN,Avoncoin,ACN,0.000722096,NaN,None,2018-02-04
ACOIN,Acoin,ACOIN,0.182166,222258.0,1220085.0,2018-02-04
ADB,adbank,ADB,0.0375622,NaN,1000000000.0,2018-02-04
ADCN,Asiadigicoin,ADCN,0.00424231,105765.0,210000000.0,2018-02-04
ADK,Aidos Kuneen,ADK,42.9628,NaN,25000000.0,2018-02-04
AI,POLY AI,AI,8.44919,NaN,29000000.0,2018-02-04
